# YouTube 자막 번역기: 영어에서 한국어로

이 노트북은 YouTube 동영상의 영어 자막을 한국어로 번역하는 도구입니다. Anthropic의 Claude AI를 사용하여 고품질의 번역을 제공합니다.

## 설정

먼저 필요한 라이브러리를 설치하고 필수 모듈을 가져옵니다.

In [ ]:
# 필요한 라이브러리 설치
!pip install requests
!pip install anthropic

# 필요한 모듈 import
import os
import sys
import anthropic
from typing import List
from google.colab import userdata, drive, files

# Google Drive 마운트
drive.mount('/content/drive')

# 상수 정의
BATCH_SIZE = 50  # 한 번에 처리할 자막의 수

## 유틸리티 함수

다음은 프로그램에서 사용되는 여러 유틸리티 함수들입니다.

In [ ]:
def get_api_key():
    """
    Google Colab의 사용자 데이터에서 Claude API 키를 가져옵니다.
    키가 없으면 오류 메시지를 출력하고 프로그램을 종료합니다.
    """
    try:
        return userdata.get('CLAUDE_API_KEY')
    except Exception as e:
        print(f"API 키 접근 오류: {e}")
        print("Google Colab에서 'ANTHROPIC_API_KEY' 시크릿을 설정했는지 확인해주세요.")
        sys.exit(1)

def read_srt_file(file_path: str) -> str:
    """
    지정된 경로에서 SRT 파일을 읽어 내용을 문자열로 반환합니다.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def write_srt_file(file_path: str, content: str):
    """
    주어진 내용을 SRT 파일로 저장합니다.
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def split_subtitles(srt_content: str) -> List[str]:
    """
    SRT 내용을 개별 자막으로 분할합니다.
    """
    return srt_content.strip().split('\n\n')

def create_batches(subtitles: List[str], batch_size: int) -> List[str]:
    """
    자막 목록을 지정된 크기의 배치로 나눕니다.
    """
    return ['\n\n'.join(subtitles[i:i+batch_size]) for i in range(0, len(subtitles), batch_size)]

## 번역 함수

다음은 실제 번역 작업을 수행하는 핵심 함수들입니다.

In [ ]:
def translate_batch(client: anthropic.Anthropic, batch: str, start_number: int) -> str:
    """
    주어진 배치의 자막을 번역합니다.
    Claude AI에 전송할 시스템 프롬프트를 생성하고, 응답을 처리합니다.
    """
    system_prompt = f"""You are tasked with translating English subtitles to Korean for a YouTube video. The subtitles are provided in SRT format. Your goal is to create high-quality Korean translations that are natural, error-free, and properly formatted.

Here are the English subtitles you need to translate:

<english_subtitles>
{{ENGLISH_SUBTITLES}}
</english_subtitles>

Follow these instructions to complete the translation:

1. Translate each subtitle from English to Korean, ensuring that the Korean translation is natural and easily understood by native speakers.

2. Numbering is performed sequentially based on the first subtitle number. Ignore other existing subtitle numbers.

3. Keep the subtitle length appropriate. If a translated subtitle becomes too long (more than two lines), split it into two or more subtitles and adjust the timing accordingly. Ensure that the numbering remains sequential.

4. Preserve any non-verbal information in parentheses, such as (laughs) or (music), translating them to Korean as well: (웃음) or (음악).

5. Use appropriate honorifics and formality levels based on the context of the video.

6. If there are any culturally specific references or idioms, adapt them to Korean equivalents that convey the same meaning.

7. For any terms that are better left in English (e.g., brand names, technical terms), use the original English term followed by a brief Korean explanation in parentheses if necessary.

Provide your translation in the following format:

<korean_subtitles>
{start_number}
[start time] --> [end time]
[Korean translation]

{start_number + 1}
[start time] --> [end time]
[Korean translation]

...
</korean_subtitles>

Remember to review your translation for accuracy, naturalness, and proper formatting before submitting your final output."""

    try:
        message = client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=4000,
            system=system_prompt,
            messages=[
                {"role": "user", "content": batch}
            ]
        )
        translated_text = message.content[0].text
        korean_subtitles = translated_text.split('<korean_subtitles>')[1].split('</korean_subtitles>')[0].strip()
        return korean_subtitles + '\n\n'
    except Exception as e:
        print(f"번역 중 오류 발생: {e}")
        raise

def renumber_subtitles(srt: str) -> str:
    """
    번역된 자막의 번호를 1부터 순차적으로 다시 매깁니다.
    """
    subtitles = srt.strip().split('\n\n')
    return '\n\n'.join(
        '\n'.join([str(i+1)] + subtitle.split('\n')[1:])
        for i, subtitle in enumerate(subtitles)
    )

def translate_subtitles(client: anthropic.Anthropic, input_file: str, output_file: str):
    """
    전체 자막 번역 프로세스를 관리합니다.
    파일을 읽고, 배치로 나누어 번역하고, 결과를 저장합니다.
    """
    srt_content = read_srt_file(input_file)
    subtitles = split_subtitles(srt_content)
    batches = create_batches(subtitles, BATCH_SIZE)

    translated_srt = ""
    subtitle_number = 1

    for i, batch in enumerate(batches, 1):
        print(f"번역 진행 중... {i}/{len(batches)}")
        try:
            translated_batch = translate_batch(client, batch, subtitle_number)
            translated_srt += translated_batch
            subtitle_number += BATCH_SIZE
        except Exception as e:
            print(f"배치 {i} 번역 중 오류 발생: {e}")
            print("다음 배치로 진행합니다.")
            continue

    translated_srt = renumber_subtitles(translated_srt)
    write_srt_file(output_file, translated_srt)
    print(f"번역 완료! 결과가 {output_file}에 저장되었습니다.")

## 메인 실행 부분

마지막으로, 프로그램의 주요 실행 부분입니다. API 키를 확인하고, 사용자로부터 입력 파일을 받아 번역을 수행하고 결과를 저장합니다.

In [ ]:
if __name__ == "__main__":
    # API 키 확인 및 클라이언트 생성
    api_key = get_api_key()
    client = anthropic.Anthropic(api_key=api_key)
    print("API 키가 성공적으로 로드되었습니다.")

    # 사용자로부터 입력 파일 업로드 받기
    print("번역할 SRT 파일을 업로드해주세요.")
    uploaded = files.upload()
    input_file_name = list(uploaded.keys())[0]
    input_file_path = f'/content/{input_file_name}'
    print(f"파일 '{input_file_name}'이(가) 업로드되었습니다.")

    # 출력 파일 이름 생성
    base, ext = os.path.splitext(input_file_name)
    output_file_name = f"{base}_ko{ext}"
    output_file_path = f'/content/{output_file_name}'

    # 번역 실행
    try:
        translate_subtitles(client, input_file_path, output_file_path)

        # 번역된 파일 다운로드
        files.download(output_file_path)

        # 번역된 파일을 Google Drive에 저장 (선택사항)
        drive_output_path = f'/content/drive/My Drive/{output_file_name}'
        !cp "{output_file_path}" "{drive_output_path}"
        print(f"번역된 파일이 Google Drive에 저장되었습니다: {drive_output_path}")

    except Exception as e:
        print(f"번역 중 오류가 발생했습니다: {e}")
        print("문제가 지속되면 API 키를 확인하고 Anthropic 지원팀에 문의하세요.")

이제 노트북을 실행할 준비가 되었습니다. 각 셀을 순서대로 실행하면 YouTube 자막을 영어에서 한국어로 번역할 수 있습니다. 번역된 파일은 자동으로 다운로드되며, Google Drive에도 저장됩니다.